In [1]:
import datetime
import pandas as pd

from evidently import DataDefinition
from evidently import Dataset
from evidently import Report
from evidently.metrics import ValueDrift, DriftedColumnsCount, MissingValueCount



In [ ]:


x_path = '../data/processed/x_train.csv'
y_path = '../data/processed/y_train.csv'   


train_data = pd.read_csv(x_path)
# y = pd.read_csv(y_path)


X = X.reset_index(drop=True)
y = y.reset_index(drop=True)



train_data = pd.concat([X, y], axis=1)




x_val_path = '../data/processed/x_test.csv'
y_val_path = '../data/processed/y_test.csv'

X_val = pd.read_csv(x_val_path)
y_val = pd.read_csv(y_val_path)


X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

val_data = pd.concat([X_val, y_val], axis=1)


train_data.columns

Index(['Age', 'Gender', 'Temperature (C)', 'Humidity', 'Wind Speed (km/h)',
       'nausea', 'joint_pain', 'abdominal_pain', 'high_fever', 'chills',
       'fatigue', 'runny_nose', 'pain_behind_the_eyes', 'dizziness',
       'headache', 'chest_pain', 'vomiting', 'cough', 'hiv_aids',
       'nasal_polyps', 'asthma', 'high_blood_pressure', 'severe_headache',
       'weakness', 'trouble_seeing', 'fever', 'body_aches', 'sore_throat',
       'sneezing', 'diarrhea', 'rapid_breathing', 'rapid_heart_rate',
       'pain_behind_eyes', 'swollen_glands', 'rashes', 'sinus_headache',
       'facial_pain', 'shortness_of_breath', 'reduced_smell_and_taste',
       'skin_irritation', 'itchiness', 'throbbing_headache', 'confusion',
       'back_pain', 'knee_ache', 'prognosis'],
      dtype='object')

In [72]:
y.prognosis.unique()

array([ 1,  8,  3,  6,  0, 10,  5,  4,  7,  2,  9])

In [29]:


x_path = '../data/processed/x_train.csv'
  


train_data = pd.read_csv(x_path)


x_val_path = '../data/processed/x_test.csv'


val_data = pd.read_csv(x_val_path)



train_data.columns

Index(['Age', 'Gender', 'Temperature (C)', 'Humidity', 'Wind Speed (km/h)',
       'nausea', 'joint_pain', 'abdominal_pain', 'high_fever', 'chills',
       'fatigue', 'runny_nose', 'pain_behind_the_eyes', 'dizziness',
       'headache', 'chest_pain', 'vomiting', 'cough', 'hiv_aids',
       'nasal_polyps', 'asthma', 'high_blood_pressure', 'severe_headache',
       'weakness', 'trouble_seeing', 'fever', 'body_aches', 'sore_throat',
       'sneezing', 'diarrhea', 'rapid_breathing', 'rapid_heart_rate',
       'pain_behind_eyes', 'swollen_glands', 'rashes', 'sinus_headache',
       'facial_pain', 'shortness_of_breath', 'reduced_smell_and_taste',
       'skin_irritation', 'itchiness', 'throbbing_headache', 'confusion',
       'back_pain', 'knee_ache'],
      dtype='object')

In [48]:
artifact_dir = Path("../data/processed")
scaler  = pickle.load(open(artifact_dir / "minmax_scaler.pkl", "rb"))
encoder = pickle.load(open(artifact_dir / "label_encoder.pkl", "rb"))
x_train_scaled = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)
x_val_scaled = pd.DataFrame(scaler.transform(x_val), columns=x_val.columns)

In [31]:


num_features = ['Age', 'Temperature (C)', 'Humidity', 'Wind Speed (km/h)']
cat_features = [col for col in train_data.columns if col not in num_features]



In [57]:
model = pickle.load(open("../models/weather_disease_model.pkl", "rb")) 

train_preds = model.predict(x_train_scaled)
x_train_scaled['prediction'] = train_preds


val_preds = model.predict(x_val_scaled)
x_val_scaled['prediction'] = val_preds

2025/07/13 13:27:18 WARNING mlflow.models.utils: Found extra inputs in the model input that are not defined in the model signature: `['prediction']`. These inputs will be ignored.


In [69]:
x_val_scaled.to_csv('../data/processed/reference.csv')

In [64]:
data_definition = DataDefinition(numerical_columns=num_features + ['prediction'], categorical_columns=cat_features)
train_dataset = Dataset.from_pandas(
    x_train_scaled,
    data_definition
)

val_dataset = Dataset.from_pandas(
    x_val_scaled,
    data_definition
)

In [65]:
report = Report(metrics=[
    ValueDrift(column='prediction'),
    DriftedColumnsCount(),
    MissingValueCount(column='prediction'),
]
)

In [66]:
snapshot = report.run(reference_data=train_dataset, current_data=val_dataset)

In [67]:
result = snapshot.dict()
result

{'metrics': [{'id': 'bfc6e8246d39abff41fc2e002575d9a3',
   'metric_id': 'ValueDrift(column=prediction)',
   'value': np.float64(0.028254353539333074)},
  {'id': '15e89f895b482f9b84ba7274ed18a106',
   'metric_id': 'DriftedColumnsCount(drift_share=0.5)',
   'value': {'count': 0.0, 'share': 0.0}},
  {'id': 'd57fce37e7dac2a48797649e0e142902',
   'metric_id': 'MissingValueCount(column=prediction)',
   'value': {'count': 0.0, 'share': np.float64(0.0)}}],
 'tests': []}

In [63]:
#prediction drift
result['metrics'][0]['value']

np.float64(0.028254353539333074)

In [19]:
#!/usr/bin/env python

import pickle
import pandas as pd
from pathlib import Path
from evidently import Report
from evidently.presets import DataDriftPreset, ClassificationPreset

artifact_dir = Path("../data/processed")
scaler  = pickle.load(open(artifact_dir / "minmax_scaler.pkl", "rb"))
encoder = pickle.load(open(artifact_dir / "label_encoder.pkl", "rb"))

x_train = pd.read_csv("../data/processed/x_train.csv")
y_train = pd.read_csv("../data/processed/y_train.csv")
x_val   = pd.read_csv("../data/processed/x_test.csv")
y_val   = pd.read_csv("../data/processed/y_test.csv")

x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)
x_val   = pd.DataFrame(scaler.transform(x_val),   columns=x_val.columns)

y_train = encoder.inverse_transform(y_train["prognosis"])
y_val   = encoder.inverse_transform(y_val["prognosis"])

ref = x_train.copy()
ref["prognosis"] = y_train
cur = x_val.copy()
cur["prognosis"] = y_val

report = Report(metrics=[DataDriftPreset(), ClassificationPreset(target="prognosis")])
report.run(reference_data=ref, current_data=cur)

ts = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
out_dir = Path("monitor_reports") / ts
out_dir.mkdir(parents=True, exist_ok=True)
report.save_html(out_dir / "weather_disease_monitoring.html")
print(out_dir)


TypeError: ClassificationPreset.__init__() got an unexpected keyword argument 'target'

In [ ]:
#!/usr/bin/env python

import pickle
import numpy as np
import pandas as pd
from pathlib import Path
import datetime
from evidently import Report
from evidently.presets import ClassificationPreset
from sklearn.preprocessing import MinMaxScaler, LabelEncoder   # only for type hints

art_dir = Path("../data/processed")

# load artefacts
scaler: MinMaxScaler = pickle.load(open(art_dir / "minmax_scaler.pkl", "rb"))
encoder: LabelEncoder = pickle.load(open(art_dir / "label_encoder.pkl", "rb"))
model = pickle.load(open("../models/weather_disease_model.pkl", "rb"))              # your trained classifier

# reference (train) and current (validation) data
x_ref = pd.read_csv(art_dir / "x_train.csv")
y_ref = pd.read_csv(art_dir / "y_train.csv")
x_cur = pd.read_csv(art_dir / "x_test.csv")
y_cur = pd.read_csv(art_dir / "y_test.csv")

# scale features
x_ref = pd.DataFrame(scaler.transform(x_ref), columns=x_ref.columns)
x_cur = pd.DataFrame(scaler.transform(x_cur), columns=x_cur.columns)

# decode ground‑truth labels
y_ref = encoder.inverse_transform(y_ref["prognosis"])
y_cur = encoder.inverse_transform(y_cur["prognosis"])

pred_ref = model.predict(x_ref)
pred_cur = model.predict(x_cur)

def is_prob(a): return np.issubdtype(a.dtype, np.floating) and np.all((a>=0)&(a<=1))
prob_mode = is_prob(pred_ref)

ref = x_ref.copy(); cur = x_cur.copy()
ref["prognosis"] = y_ref;  cur["prognosis"] = y_cur

cm = ColumnMapping(target="prognosis")      # <-- base mapping

if prob_mode:                               # probabilities
    ref["prediction_proba"] = pred_ref
    cur["prediction_proba"] = pred_cur
    cm.prediction_proba = "prediction_proba"
else:                                       # class labels
    ref["prediction"] = pred_ref
    cur["prediction"] = pred_cur
    cm.prediction = "prediction"

report = Report(metrics=[ClassificationPreset(probas_threshold=0.5)])

report.run(reference_data=ref, current_data=cur,)  # <-- pass mapping

ts = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
out = Path("monitor_reports")/ts; out.mkdir(parents=True, exist_ok=True)
report.save_html(out/"weather_disease_classification.html")
print(out)  

ImportError: cannot import name 'ColumnMapping' from 'evidently' (/Users/user1/Data/mlops/weather_health/.venv/lib/python3.10/site-packages/evidently/__init__.py)